In [30]:
import pandas as pd
import folium
from folium.plugins import HeatMap

In [31]:
df = pd.read_csv('King_County_House_prices_dataset.csv', parse_dates=True)
df['zipcode'].unique()

array([98178, 98125, 98028, 98136, 98074, 98053, 98003, 98198, 98146,
       98038, 98007, 98115, 98107, 98126, 98019, 98103, 98002, 98133,
       98040, 98092, 98030, 98119, 98112, 98052, 98027, 98117, 98058,
       98001, 98056, 98166, 98023, 98070, 98148, 98105, 98042, 98008,
       98059, 98122, 98144, 98004, 98005, 98034, 98075, 98116, 98010,
       98118, 98199, 98032, 98045, 98102, 98077, 98108, 98168, 98177,
       98065, 98029, 98006, 98109, 98022, 98033, 98155, 98024, 98011,
       98031, 98106, 98072, 98188, 98014, 98055, 98039])

In [32]:
m = folium.Map([47.608013, -122.335167], zoom_start=9)

In [33]:
m

In [4]:
m.add_children(HeatMap(df[['lat', 'long']], radius=15))

//anaconda3/envs/nf/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method `add_children` is deprecated. Please use `add_child` instead.
  """Entry point for launching an IPython kernel.


In [10]:
mean_price_per_zip = df.groupby('zipcode')[['price']].mean()
mean_price_per_zip.reset_index(inplace=True)
mean_price_per_zip['zipcode'] = mean_price_per_zip['zipcode'].astype('str')
mean_price_per_zip2=pd.DataFrame(data=[mean_price_per_zip.iloc[0]], columns=mean_price_per_zip.columns)

In [18]:
sum_price_per_zip = df.groupby('zipcode')[['price']].sum()
sum_price_per_zip.reset_index(inplace=True)
sum_price_per_zip['zipcode'] = sum_price_per_zip['zipcode'].astype('str')
sum_price_per_zip2=pd.DataFrame(data=[sum_price_per_zip.iloc[0]], columns=sum_price_per_zip.columns)

In [63]:
median_price_per_zip = df.groupby('zipcode')[['price']].median()
median_price_per_zip.reset_index(inplace=True)
median_price_per_zip['zipcode'] = median_price_per_zip['zipcode'].astype('str')

In [66]:
count_per_zip = df.groupby('zipcode')[['price']].count()
count_per_zip.reset_index(inplace=True)
count_per_zip['zipcode'] = count_per_zip['zipcode'].astype('str')

In [71]:
grade_per_zip = df.groupby('zipcode')[['grade']].mean()
grade_per_zip.reset_index(inplace=True)
grade_per_zip['zipcode'] = grade_per_zip['zipcode'].astype('str')

In [4]:
import json
with open('Zipcodes_for_King_County.geojson') as file:
    data = json.load(file)    


feat = data['features']
ziplist = []
ziplist2 = []
for i in range(len(feat)):
    cur = feat[i]['properties']['ZIPCODE']
    if cur in list(mean_price_per_zip['zipcode'].unique()):
        if cur not in ziplist2:
            ziplist2.append(cur)
            ziplist.append(feat[i])
            
zipped_json = dict.fromkeys(['type', 'features'])
zipped_json['type'] = 'FeatureCollection'
zipped_json['features'] = ziplist

In [9]:
with open('zc_for_kc_short.geojson', 'w') as f:
    f.write(json.dumps(zipped_json, sort_keys=False, indent=4, separators=(',', ': ')))

In [10]:
with open('zc_for_kc_short.geojson') as file:
    data_short = json.load(file) 

len(data['features'])

204

In [11]:
import geopandas as gpd

fname = 'zc_for_kc_short.geojson'

df2 = gpd.read_file(fname)
df2.sort_values('ZIPCODE', inplace=True)
df2.reset_index(inplace=True)
df2.head()
#df2.to_dict()
data_final = json.load(df2.to_json())

AttributeError: 'str' object has no attribute 'read'

In [12]:
with open('test.geojson') as file:
    test = json.load(file)


In [13]:
test['features'][0]['properties']['ZIPCODE'] == mean_price_per_zip['zipcode'][0]
#mean_price_per_zip

True

In [11]:
import folium
from folium.features import Choropleth

In [25]:
bins = pd.cut(sum_price_per_zip['price'], bins=4)
bins.dtypes

CategoricalDtype(categories=[(15826011.685, 119684368.75], (119684368.75, 223128947.5], (223128947.5, 326573526.25], (326573526.25, 430018105.0]]
              ordered=True)

In [70]:
geo = 'cleaned_geodata.json'
m3 = folium.Map([47.608013, -122.335167], zoom_start=9)
folium.Choropleth(
    geo_data = geo,
    data = median_price_per_zip,
    #data = sum_price_per_zip,
    columns = ['zipcode', 'price'],
    key_on = 'feature.properties.ZIP',
    fill_color = 'YlOrRd',
    fill_opacity=0.7,
    line_opacity=.3,
    name='choropleth',
    legend_name = 'Median sale price',
    #bins=[15826011.685, 119684368.75, 223128947.5, 326573526.25, 430018105.0]
    ).add_to(m3)
m3

In [69]:


geo = 'cleaned_geodata.json'
m_count = folium.Map([47.608013, -122.335167], zoom_start=9)
folium.Choropleth(
    geo_data = geo,
    data = count_per_zip,
    #data = sum_price_per_zip,
    columns = ['zipcode', 'price'],
    key_on = 'feature.properties.ZIP',
    fill_color = 'YlOrRd',
    fill_opacity=0.7,
    line_opacity=.3,
    name='choropleth',
    legend_name = '# of house sales',
    #bins=[15826011.685, 119684368.75, 223128947.5, 326573526.25, 430018105.0]
    ).add_to(m_count)
m_count

In [56]:
spread_per_zip = df.groupby('zipcode')[['price']].max() - df.groupby('zipcode')[['price']].min()
spread_per_zip.reset_index(inplace=True)
spread_per_zip['zipcode'] = spread_per_zip['zipcode'].astype('str')

m_spread = folium.Map([47.608013, -122.335167], zoom_start=9)


folium.Choropleth(
    geo_data = geo,
    #data = mean_price_per_zip,
    data = spread_per_zip,
    columns = ['zipcode', 'price'],
    key_on = 'feature.properties.ZIP',
    fill_color = 'YlOrRd',
    fill_opacity=0.7,
    line_opacity=.3,
    name='choropleth',
    legend_name = 'Spread',
    #bins=[15826011.685, 119684368.75, 223128947.5, 326573526.25, 430018105.0]
    ).add_to(m_spread)

folium.LayerControl().add_to(m_spread)
m_spread

In [73]:

m_grade = folium.Map([47.608013, -122.335167], zoom_start=9)


folium.Choropleth(
    geo_data = geo,
    #data = mean_price_per_zip,
    data = grade_per_zip,
    columns = ['zipcode', 'grade'],
    key_on = 'feature.properties.ZIP',
    fill_color = 'YlOrRd',
    fill_opacity=0.7,
    line_opacity=.3,
    name='choropleth',
    legend_name = 'Spread',
    #bins=[15826011.685, 119684368.75, 223128947.5, 326573526.25, 430018105.0]
    ).add_to(m_grade)

folium.LayerControl().add_to(m_grade)
m_grade

In [114]:
df['grade_perprice'] = df['grade']/df['price']
grade_perprice_per_zip = df.groupby('zipcode')[['grade_perprice']].mean()
grade_perprice_per_zip.reset_index(inplace=True)
grade_perprice_per_zip['zipcode'] = grade_perprice_per_zip['zipcode'].astype('str')

In [88]:
m_grade_perprice = folium.Map([47.608013, -122.335167], zoom_start=9)

folium.Choropleth(
    geo_data = geo,
    #data = mean_price_per_zip,
    data = grade_perprice_per_zip,
    columns = ['zipcode', 'grade_perprice'],
    key_on = 'feature.properties.ZIP',
    fill_color = 'YlOrRd',
    fill_opacity=0.7,
    line_opacity=.3,
    name='choropleth',
    legend_name = 'Spread',
    #bins=[15826011.685, 119684368.75, 223128947.5, 326573526.25, 430018105.0]
    ).add_to(m_grade_perprice)

folium.LayerControl().add_to(m_grade_perprice)
m_grade_perprice

In [116]:
df['sqftliving_perprice'] = df['sqft_living']/df['price']
sqftliving_per_zip = df.groupby('zipcode')[['sqftliving_perprice']].mean()
sqftliving_per_zip.reset_index(inplace=True)
sqftliving_per_zip['zipcode'] = sqftliving_per_zip['zipcode'].astype('str')

df['grade_perprice'] = df['grade']/df['price']
grade_perprice_per_zip = df.groupby('zipcode')[['grade_perprice']].mean()
grade_perprice_per_zip.reset_index(inplace=True)
grade_perprice_per_zip['zipcode'] = grade_perprice_per_zip['zipcode'].astype('str')

df['bathrooms_perusd'] = df['bathrooms']/df['price']
bathrooms_perusd_per_zip = df.groupby('zipcode')[['bathrooms_perusd']].mean()
bathrooms_perusd_per_zip.reset_index(inplace=True)
bathrooms_perusd_per_zip['zipcode'] = bathrooms_perusd_per_zip['zipcode'].astype('str')

m = folium.Map([47.608013, -122.335167], zoom_start=9)

folium.Choropleth(
    geo_data = geo,
    #data = mean_price_per_zip,
    data = sqftliving_per_zip,
    columns = ['zipcode', 'sqftliving_perprice'],
    key_on = 'feature.properties.ZIP',
    fill_color = 'YlOrRd',
    fill_opacity=0.6,
    line_opacity=.3,
    legend_name = 'Living space per USD',
    name = 'sqftliving_perusd',
    #bins=[15826011.685, 119684368.75, 223128947.5, 326573526.25, 430018105.0]
    ).add_to(m)



folium.Choropleth(
    geo_data = geo,
    #data = mean_price_per_zip,
    data = grade_perprice_per_zip,
    columns = ['zipcode', 'grade_perprice'],
    key_on = 'feature.properties.ZIP',
    fill_color = 'YlOrRd',
    fill_opacity=0.6,
    line_opacity=.3,
    legend_name = 'Grade points per USD',
    #bins=[15826011.685, 119684368.75, 223128947.5, 326573526.25, 430018105.0]
    name = 'gradepoints_perusd',
    
    ).add_to(m)

folium.Choropleth(
    geo_data = geo,
    #data = mean_price_per_zip,
    data = bathrooms_perusd_per_zip,
    columns = ['zipcode', 'bathrooms_perusd'],
    key_on = 'feature.properties.ZIP',
    fill_color = 'YlOrRd',
    fill_opacity=0.6,
    line_opacity=.3,
    legend_name = 'Bathrooms per USD',
    #bins=[15826011.685, 119684368.75, 223128947.5, 326573526.25, 430018105.0]
    name = 'bathrooms_perusd',
    
    ).add_to(m)

folium.LayerControl().add_to(m)
m

In [145]:
m4 = folium.Map([47.608013, -122.335167], zoom_start=9)
#m4.add_child(HeatMap(df[['lat', 'long']][df['waterfront'].isna()], radius=6, blur=1))
m4.add_child(HeatMap(df[['lat', 'long']][df['waterfront'] == 1.0], radius=8, blur=1, name='Waterfront houses'))

In [50]:
df[df['waterfront'] == 1.0]['price'].describe()

count    1.460000e+02
mean     1.717215e+06
std      1.145385e+06
min      2.850000e+05
25%      8.275000e+05
50%      1.510000e+06
75%      2.282500e+06
max      7.060000e+06
Name: price, dtype: float64

In [143]:
print('Living area (in sqft) per 100,000 USD sale price for waterfront property: ', df[df['waterfront'] == 1.0]['sqftliving_perprice'].median()*100000)
print('Living area (in sqft) per 100,000 USD sale price for non-waterfront property: ', df[df['waterfront'] != 1.0]['sqftliving_perprice'].median()*100000)

Living area (in sqft) per 100,000 USD sale price for waterfront property:  193.42879256965944
Living area (in sqft) per 100,000 USD sale price for non-waterfront property:  409.87654320987656


In [142]:
df[df['waterfront'] == 1.0]['bathrooms_perusd'].mean()*100000
print('Bathrooms per 100,000 USD sale price for waterfront property: ', df[df['waterfront'] == 1.0]['bathrooms_perusd'].median()*100000)
print('Bathrooms per 100,000 USD sale price for non-waterfront property: ', df[df['waterfront'] != 1.0]['bathrooms_perusd'].median()*100000)

Bathrooms per 100,000 USD sale price for waterfront property:  0.16666666666666666
Bathrooms per 100,000 USD sale price for non-waterfront property:  0.430622009569378


In [141]:
print('Grade points per 100,000 USD sale price for waterfront property: ', df[df['waterfront'] == 1.0]['bathrooms_perusd'].median()*100000)
print('Grade points per 100,000 USD sale price for non-waterfront property: ', df[df['waterfront'] != 1.0]['bathrooms_perusd'].median()*100000)

Grade points per 100,000 USD sale price for waterfront property:  0.16666666666666666
Grade points per 100,000 USD sale price for non-waterfront property:  0.430622009569378
